# Evolutionary Computation Assignment 4

##### Solutions checked by solution checker

- Krzysztof Szala 144571
- Vadym Repetskyi 155610

# Problem description

#### The use of candidate moves in local search
- The goal of the task is to improve the time efficiency of the steepest local search with the use of
candidate moves using the neighborhood, which turned out to be the best in the previous
assignment,
- As candidate moves, we use moves that introduce at least one candidate edge to the solution. We
define the candidate edges by determining for each vertex 10 other “nearest” vertices (“nearest”
taking into account sum of the edge length and vertex cost). This parameter (10) can also be selected
experimentally to obtain the best results.
- Note that both in the case of intra-route moves (e.g. two-edges exchange) and inter-route moves
(nodes-exchange with one selected one not selected) we should ensure that at least one candidate
edge is introduced to the solution. In particular, it is not correct to exchange a selected node with
one of its nearest neighbors without adding to the solution at least one candidate edge.
- Note that there are in both cases two moves that introduce a given candidate edge.
- As starting solutions use random solutions.
- As baseline report also results of the steepest local search with random starting solutions without
these mechanisms.
- Computational experiment: Run each of the methods 200 times.

In [ ]:
from utils import TspInstance, random_solution, local_search
import pandas as pd
import numpy as np

# Pseudocodes

## Candidate Moves

1. Generate Random Solution
2. For each vertex x
    1. For each vertex y from the list of the closes vertices to vertex x -> Evaluate all (two) moves involving the addition of edge (x,y) and the removal of one of the edges adjacent to x

In [ ]:
def single_inter_move(tsp, solution, to_unselect, to_select):
    solution = solution.copy()
    insert_index = np.where(solution == to_unselect)[0][0]

    prev = solution[insert_index - 1]
    next = solution[(insert_index + 1) % len(solution)]

    cost_change = (
        tsp.node_costs[to_select]
        + tsp.distance_matrix[prev, to_select]
        + tsp.distance_matrix[to_select, next]
        - tsp.node_costs[to_unselect]
        - tsp.distance_matrix[prev, to_unselect]
        - tsp.distance_matrix[to_unselect, next]
    )
    
    if to_unselect is not None:
        solution[np.where(solution == to_unselect)] = to_select
    
    return solution, cost_change

In [ ]:
def single_intra_move(tsp, solution, a, b):
    solution = solution.copy()
    index_a = np.where(solution == a)[0][0]
    index_b = np.where(solution == b)[0][0]

    a_prev = solution[index_a - 1]
    b_prev = solution[index_b - 1]
    if index_a - 1 < 0:
        a_prev = solution[len(solution) - 1]
    if index_b - 1 < 0:
        b_prev = solution[len(solution) - 1]

    a_next = solution[(index_a + 1) % len(solution)]
    b_next = solution[(index_b + 1) % len(solution)]

    cost_change = (
        tsp.distance_matrix[a_prev, b]
        + tsp.distance_matrix[b, a_next]
        + tsp.distance_matrix[b_prev, a]
        + tsp.distance_matrix[a, b_next]
        - tsp.distance_matrix[a_prev, a]
        - tsp.distance_matrix[a, a_next]
        - tsp.distance_matrix[b_prev, b]
        - tsp.distance_matrix[b, b_next]
    )

    if index_a < index_b:
        subsequence = solution[(index_a + 1):index_b+1][::-1]
        solution[(index_a + 1):index_b+1] = subsequence
    else:
        subsequence = solution[(index_b + 1):index_a+1][::-1]
        solution[(index_b + 1):index_a+1] = subsequence

    return solution, cost_change

In [ ]:
def candidate_list(tsp: TspInstance, start_node: int):
    
    solution = random_solution(tsp, start_node)
    
    for node_x in solution:
        # Find 10 the closest vertices to the current node
        closest_vertices = np.argsort(tsp.distance_matrix[node_x] + tsp.node_costs)
        closest_vertices = closest_vertices[closest_vertices != node_x][:10]
        
        best_cost_change = 0
        best_solution    = None
        for node_y in closest_vertices:
            # Evaluate inter and intra route moves
            if node_y in solution:
                solution, cost_change = single_intra_move(tsp, solution, node_x, node_y)
            else:
                solution, cost_change = single_inter_move(tsp, solution, node_x, node_y)
                
            if cost_change < best_cost_change:
                best_cost_change = cost_change
                best_solution = solution.copy()
                    
        if best_solution is not None:
            solution = best_solution
        
    return solution

# Experiments

In [ ]:
columns = []
experiments = []

for instance in (TspInstance("TSPA.csv"), TspInstance("TSPB.csv")):
    column = f"{instance.file_path[3]}"
    experiments.append(
        instance.run_experiments(
            local_search, initial_solution_getter=random_solution, intra_type="edges", steepest=True
        )
    )
    instance.plot(experiments[-1][-1], column)

# Conlusions